In [ ]:
from notebook import Notebook
import os
import shutil
from string import Template
import sys
from io import StringIO
from enum import Enum
import logging
import re

In [ ]:
root = logging.getLogger()
root.setLevel('INFO')

handler = logging.StreamHandler(sys.stdout)
handler.setLevel('INFO')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)


parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

logging.info('Logging parameters: ' + ''.join(parameters))
for parameter in parameters:
    logging.info('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
#version="0.1n"
#notebook_path = '../component-library/input/input-url.ipynb'

In [ ]:
nb = Notebook(notebook_path)

In [ ]:
name = nb.get_name()
description = nb.get_description() + " CLAIMED v"+ version
inputs = nb.get_inputs()
outputs = nb.get_outputs()
requirements = nb.get_requirements()

In [ ]:
print(name)
print(description)
print(inputs)
print(outputs)
print(requirements)

In [ ]:
!echo {notebook_path}

In [4]:
def check_variable(var_name):
    return var_name in locals() or var_name in globals()


In [ ]:
#target_code = notebook_path.replace('.ipynb','.py').split('/')[-1:][0]

target_code = notebook_path.split('/')[-1:][0]
shutil.copy(notebook_path,target_code)
if check_variable('additionl_files'):
    if additionl_files.startswith('['):
        local_path = 'additionl_file_path'
        if not os.path.exists(local_path):
            os.makedirs(local_path)
        additionl_files_local = local_path
        additionl_files=additionl_files[1:-1].split(',')
        print('Additional files to add to container:')
        for additionl_file in additionl_files:
            print(additionl_file)
            shutil.copy(additionl_file, additionl_files_local)
        print(os.listdir(local_path))
    else:
        additionl_files_local = additionl_files.split('/')[-1:][0]
        shutil.copy(additionl_files,additionl_files_local)
else:
    additionl_files_local=target_code #hack

In [ ]:
import re

file = target_code

# they can be also raw string and regex
textToSearch = r'!pip' 
textToReplace = '#!pip'

# read and replace
with open(file, 'r') as fd:
    text, counter = re.subn(textToSearch, textToReplace, fd.read(), re.I)

# check if there is at least a  match
if counter > 0:
    # edit the file
    with open(file, 'w') as fd:
        fd.write(text)

In [ ]:
requirements_docker = list(map(lambda s: 'RUN '+s, requirements))

In [ ]:
requirements_docker = '\n'.join(requirements_docker)
docker_file = f"""
FROM registry.access.redhat.com/ubi8/python-39 
USER root
RUN dnf install -y java-11-openjdk
USER default
RUN pip install ipython==8.6.0 nbformat==5.7.0
{requirements_docker}
ADD {additionl_files_local} /opt/app-root/src/
ADD {target_code} /opt/app-root/src/
USER root
RUN chmod -R 777 /opt/app-root/src/
USER default
CMD ["ipython", "/opt/app-root/src/{target_code}"]
"""
with open("Dockerfile", "w") as text_file:
    text_file.write(docker_file)
!cat Dockerfile

In [ ]:
!docker build --platform=linux/amd64 -t `echo claimed-{name}:{version}` .
!docker tag  `echo claimed-{name}:{version}` `echo {repository}/claimed-{name}:{version}`
!docker tag  `echo claimed-{name}:{version}` `echo {repository}/claimed-{name}:latest`
!docker push `echo {repository}/claimed-{name}:{version}`
!docker push `echo {repository}/claimed-{name}:latest`

In [ ]:
parameter_type = Enum('parameter_type', ['INPUT', 'OUTPUT'])

def get_component_interface(parameters, type : parameter_type):
    template_string = str()
    for parameter_name, parameter_options in parameters.items():
        default = ''
        if parameter_options['default'] is not None and type == parameter_type.INPUT:
                default = f", default: {parameter_options['default']}"
        template_string += f"- {{name: {parameter_name}, type: {parameter_options['type']}, description: {parameter_options['description']}{default}}}"
        template_string += '\n'
    return template_string

In [ ]:
def get_output_name():
    for output_key, output_value in outputs.items():
        return output_key

In [ ]:

def get_input_for_implementation():
    with StringIO() as inputs_str:
        for input_key, input_value in inputs.items():
            t = Template("        - {inputValue: $name}")
            print(t.substitute(name=input_key), file=inputs_str)
        return inputs_str.getvalue()    

In [ ]:
def get_parameter_list():
    return_value = str()
    index = 0
    for output_key, output_value in outputs.items():
        return_value = return_value + output_key + '="${' + str(index) + '}" '
        index = index + 1
    for input_key, input_value in inputs.items():
        return_value = return_value + input_key + '="${' + str(index) + '}" '
        index = index + 1
    return return_value      

In [ ]:

t = Template('''name: $name
description: $description

inputs:
$inputs

implementation:
    container:
        image: $container_uri:$version
        command:
        - sh
        - -ec
        - |
          ipython $call
$input_for_implementation''')
yaml = t.substitute(
            name=name,
            description=description,
            inputs=get_component_interface(inputs, parameter_type.INPUT),
            container_uri=f"{repository}/claimed-{name}",
            version=version,
            outputPath=get_output_name(),
            input_for_implementation=get_input_for_implementation(),
            call=f'./{target_code} {get_parameter_list()}' 
        )
print(yaml)

In [ ]:
target_yaml_path = notebook_path.replace('.ipynb','.yaml')

with open(target_yaml_path, "w") as text_file:
    text_file.write(yaml)

In [ ]:
env_entries = []
for input_key, _ in inputs.items():
    env_entry = f"        - name: {input_key}\n          value: value_of_{input_key}"
    env_entries.append(env_entry)
    env_entries.append('\n')
env_entries.pop(-1)
env_entries = ''.join(env_entries)


In [ ]:
job_yaml = f'''apiVersion: batch/v1
kind: Job
metadata:
  name: {name}
spec:
  template:
    spec:
      containers:
      - name: {name}
        image: {repository}/claimed-{name}:{version}
        command: ["/opt/app-root/bin/ipython","/opt/app-root/src/{target_code}"]
        env:
{env_entries}
      restartPolicy: OnFailure'''


print(job_yaml)

In [ ]:
target_job_yaml_path = notebook_path.replace('.ipynb','.job.yaml')

with open(target_job_yaml_path, "w") as text_file:
    text_file.write(job_yaml)